In [ ]:
import tensorflow as tf
import numpy as np
import os
from google.colab import drive

In [ ]:
path_to_file = tf.keras.utils.get_file('dpcq.txt', 'https://www.txt990.org/api/txt_down.php?articleid=297&articlename=%E6%96%97%E7%A0%B4%E8%8B%8D%E7%A9%B9')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [ ]:
!sudo apt install -y fluidsynth

!pip install --upgrade pyfluidsynth

!pip install pretty_midi

In [ ]:
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

_SAMPLING_RATE = 16000

sample_file = tf.keras.utils.get_file('chpn_op23.mid','http://www.piano-midi.de/midis/chopin/chpn_op23.mid')
sample_file1 = tf.keras.utils.get_file('chp_op31_format0.mid','http://www.piano-midi.de/midis/chopin/chp_op31_format0.mid')
sample_file2 = tf.keras.utils.get_file('chpn_op53_format0.mid','http://www.piano-midi.de/midis/chopin/chpn_op53_format0.mid')

bach = tf.keras.utils.get_file('bwv988.mid','http://www.jsbach.net/midi/bwv988/bwv988.mid')

pm = pretty_midi.PrettyMIDI(sample_file)
pm1 = pretty_midi.PrettyMIDI(sample_file1)
pm2 = pretty_midi.PrettyMIDI(sample_file2)
pmbach = pretty_midi.PrettyMIDI(bach)

def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [ ]:
#raw_notes = [midi_to_notes(sample_file),midi_to_notes(sample_file1),midi_to_notes(sample_file2),]
raw_notes = [midi_to_notes(bach),midi_to_notes(sample_file),midi_to_notes(sample_file1),midi_to_notes(sample_file2)]
print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
text = ''
for rawn in raw_notes:
  for i, note in rawn.iterrows():
    for times in range(round(note['duration']/0.07)):
      text += chr(int(note['pitch']))

In [ ]:
vocab = sorted(set(text))

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

sequences = ids_dataset.batch(151, drop_remainder=True)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 256

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 1024

# Number of RNN units
rnn_units = 2048
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

In [ ]:
import os
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, run_eagerly=True)
checkpoint_dir = './music'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

In [ ]:
for i in range(100):
  model.fit(dataset, epochs=1, callbacks=[checkpoint_callback])

In [ ]:
 class OneStep(tf.keras.Model):
     def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
         super().__init__()
             self.temperature = temperature
                 self.model = model
                     self.chars_from_ids = chars_from_id. class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    
    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
states = None
next_char = tf.constant(['0'])
#result = [next_char]

raw = []
for n in range(2000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  #result.append(next_char)
  raw.append(ord(tf.strings.join(next_char).numpy().decode('utf-8')))
#result = tf.strings.join(result)
#print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

In [ ]:
def notes_to_midi(
  raw,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  start,end = 0,0
  for i in range(len(raw)-1):
    if raw[i]==raw[i+1]:
      end += 0.2
    else:
      note = pretty_midi.Note(
          velocity=velocity,
          pitch=int(raw[i]),
          start=start,
          end = end,
      )
      instrument.notes.append(note)
      start = end
  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

In [ ]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=60):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [ ]:
example_file = 'example.midi'
example_pm = notes_to_midi(
    raw, out_file=example_file, instrument_name=instrument_name)
display_audio(example_pm)

In [ ]:
ori = []
for chr in text:
  ori.append(ord(chr))
example_pm = notes_to_midi(
    ori, out_file=example_file, instrument_name=instrument_name)
display_audio(example_pm)